### Code to Calculate Free Energy of Releasing Distance Restraint

In [10]:
# Code closely based on restraints.py in
# Yank https://github.com/choderalab/yank 
# (for releasing Boresch restraints)

import numpy as np
import os
from math import pi, sin, log
import scipy.integrate
from Sire import Units

# Constants
v0 = ((Units.meter3/1000)/Units.mole.value()).value() # A^3, the standard state volume
R = Units.gasr # kcal mol-1, the molar gas constant
T = 298.15 # K

def numerical_distance_integrand(r, r0, kr):
    """Integrand for harmonic distance restraint. Domain is on [0, infinity], 
    but this will be truncated to [0, 8 RT] for practicality.

    Args:
        r (float): Distance to be integrated, in Angstrom 
        r0 (float): Equilibrium distance, in Angstrom
        kr (float): Force constant, in kcal mol-1 A-2

    Returns:
        float: Value of integrand
    """
    return (r**2)*np.exp(-(kr*(r-r0)**2)/(2*R*T))

def get_correction(r0, kr):
    """Get the free energy of releasing the harmonic distance restraint.
    Domain is on [0, infinity], but this will be truncated to [0, 8 RT] for practicality.
    Args:
        r0 (float): Equilibrium distance, in Angstrom
        kr (float): Force constant, in kcal mol-1 A-2
    Returns:
        float: Free energy of releasing the restraint
    """
    dist_at_8RT = 4*np.sqrt((R*T)/kr) # Dist. which gives restraint energy = 8 RT
    r_min = max(0, r0-dist_at_8RT)
    r_max = r0 + dist_at_8RT
    integrand = lambda r: numerical_distance_integrand(r, r0, kr)
    z_r = scipy.integrate.quad(integrand, r_min, r_max)[0]
    dg = -R*T*log(v0/(4*np.pi*z_r))

    return dg

### Free Energy of Releasing the Restraint if r0 == 0

In [11]:
cor_r0 = get_correction(0, 10)
print(f"Free energy of releasing restraint when r_0 = 0: {cor_r0:.2f} kcal mol-1")

Free energy of releasing restraint when r_0 = 0: -5.27 kcal mol-1


This is in excellent agreement with the correction shown in Table 1 for the correction with no orientational restraints (-5.26 kcal mol-1)

### Free Energy of Releasing the Restraint if r0 = 3.48 A

In [17]:
cor_r0 = get_correction(3.48, 10)
print(f"Free energy of releasing restraint when r_0 = 3.48 A: {cor_r0:.2f} kcal mol-1")

Free energy of releasing restraint when r_0 = 3.48 A: -1.71 kcal mol-1


For the simulation without orientational restraints and with 1 ns sampling time, the overall free energy is 5.95 +/- 0.09 kcal mol-1. If the above correction is used instead, this gives 5.95 +5.26 - 1.71 = 9.50 +/- 0.09 kcal mol-1, much closer to the answer obtained with orientational restraints (10.03 +/- 0.05 kcal mol-1).